# Exception in bultin generator - confusing traceback
* https://discuss.python.org/t/misleading-error-message-shows-the-wrong-code/19777?u=vbrozik

In [1]:
a = [1, 2]
b = [3, 4]
diffs = filter(None, map(a, b))
next(diffs, len(a) - len(b))

TypeError: 'list' object is not callable

There is no `list` object at the line 4. The exception happened inside the `map()` function but its frame was completely omitted from the traceback. The normal Python traceback:
``` python
Traceback (most recent call last):
  File "/python/test_generator_exception_orig_map.py", line 4, in <module>
    next(diffs, len(a) - len(b))
TypeError: 'list' object is not callable
```

In [7]:
def map(function, *iterable):
    for items in zip(*iterable):
        yield function(*items)

a = [1, 2]
b = [3, 4]
diffs = filter(None, map(a, b))
next(diffs, len(a) - len(b))

TypeError: 'list' object is not callable

In [2]:
def map(function, *iterables):
    return (function(*items) for items in zip(*iterables))

a = [1, 2]
b = [3, 4]
diffs = filter(None, map(a, b))
next(diffs, len(a) - len(b))

TypeError: 'list' object is not callable

The normal Python traceback - now not confusing:
``` python
Traceback (most recent call last):
  File "/python/test_generator_exception.py", line 7, in <module>
    next(diffs, len(a) - len(b))
  File "/python/test_generator_exception.py", line 2, in <genexpr>
    return (function(*items) for items in zip(*iterables))
            ^^^^^^^^^^^^^^^^
TypeError: 'list' object is not callable
```

Solution ideas:
* Virtual frame for non-python generators.
* Extend the exception message by information:
  * exception happened inside `__builtins__.map()`
  * exception happened when processing the first argument
* Partial solution: Mark the iterator which caused the exception.